## 1. you must setting with high-memory mode (25GB)
## 2. if you use GPU, you must use colab PRO
## 3. before unzip, do not store over 5.1GB data
## 4. upload your API kaggle.json  on  default folder (content/) 
> #### maybe, you have to launch once more(?)
## after preprocessing, you can freely restart you session.
## actually numpy memmap reading is "very" slow(250s reading one cycle of train dataset).
> #### So most of time with small model, P100 and V100 have small difference.
> #### but, use V100 if you can

### download ~ 15min
### unzip & processing ~ 1.1 hour
### output shape = (None, 3, 273, 256)
> ### if you want to use as (None, 273, 256, 3), 
> > ### than use tf.keras.layers.Permute((2,3,1))
## if you restart, do not change validation_rate

In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
import keras.layers as L
import math, random
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing import image
from random import shuffle
from sklearn.model_selection import train_test_split
import plotly.express as px
import seaborn as sns
import os, gc, time
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
!pip install -q -U tensorflow-addons
import tensorflow_addons as tfa

from tensorflow.keras.mixed_precision import experimental as mixed_precision#!
policy = mixed_precision.Policy('mixed_float16')#!
mixed_precision.set_policy(policy)#!

random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)


# preprocessing

In [ ]:
def id_to_path(idxs,train=True):
    for idx in idxs:
        path = ''
        if train:
            folder = 'train/'
        else:
            folder = 'test/'
        path+=folder+idx[0]+'/'+idx+'.npy'
        if os.path.exists(path): yield path
def as_numpy_dataset(idxs, is_train):
  loading = [np.load(path, mmap_mode = 'r') for path in id_to_path(idxs, is_train)]
  print(loading[0].dtype, loading[0].shape )
  loading = [I[::2] for I in loading]
  tmp_result = np.asarray(loading)
  print(tmp_result.dtype)
  if tmp_result.dtype == np.float16: return tmp_result
  return tmp_result.astype(np.float16)

In [ ]:
validation_rate = 0.118 # do not change after build dataset 
try:
  sample_submission = pd.read_csv('sample_submission.csv')
  test_idx = sample_submission['id'].values
  tests_DT = np.memmap('test_mmap', dtype = 'float16', mode = 'r', shape = (len(test_idx), 3, 273, 256) )
except:
  !sudo mv kaggle.json  ../root/.kaggle/kaggle.json 
  !pip install --upgrade --force-reinstall --no-deps kaggle
  !kaggle competitions download -c seti-breakthrough-listen
  
  !unzip seti-breakthrough-listen.zip  -x train/*
  sample_submission = pd.read_csv('sample_submission.csv')
  test_idx = sample_submission['id'].values
  tmp_DT = as_numpy_dataset(test_idx, is_train  = False)
  tests_DT = np.memmap('test_mmap', dtype = tmp_DT.dtype, mode = 'w+', shape = tmp_DT.shape)
  tests_DT[:] = tmp_DT[:]
  tests_DT.flush()
  del tmp_DT
  !sudo rm -r test
  
try:
  train_labels =pd.read_csv('train_labels.csv')
  train_idx =  train_labels['id'].values
  y = train_labels['target'].values
  x_train,x_valid, y_train,y_valid = train_test_split(train_idx,y,test_size=validation_rate,random_state=42)
  x_train, y_train = [np.asarray(I) for I in zip(*sorted(list(zip(x_train, y_train))))]
  x_valid, y_valid = [np.asarray(I) for I in zip(*sorted(list(zip(x_valid, y_valid))))]

  valid_DT = np.memmap('valid_mmap', dtype = 'float16', mode = 'r', shape = (y_valid.shape[0], 3, 273, 256))
  valid_DT = valid_DT, y_valid

  train_DT = np.memmap('train_mmap', dtype = 'float16', mode = 'r', shape = (y_train.shape[0], 3, 273, 256))
  #train_DT = np.array(train_DT), y_train
  train_DT = train_DT, y_train
except:
  #train_idx가 정렬되어있기 때문에 분리처리후 병합
  !sudo rm -r train
  train_labels = pd.read_csv('train_labels.csv')
  train_idx =  train_labels['id'].values
  y = train_labels['target'].values
  x_train,x_valid, y_train,y_valid = train_test_split(train_idx,y,test_size=validation_rate,random_state=42)
  x_train, y_train = [np.asarray(I) for I in zip(*sorted(list(zip(x_train, y_train))))]
  x_valid, y_valid = [np.asarray(I) for I in zip(*sorted(list(zip(x_valid, y_valid))))]
  

  !unzip seti-breakthrough-listen.zip train/[0-9]/*
  tmp_train_DT = as_numpy_dataset(x_train, is_train  = True)

  gc.collect();time.sleep(5)
  valid_DT = as_numpy_dataset(x_valid, is_train  = True)
  !sudo rm -r train 



  !unzip seti-breakthrough-listen.zip train/[^0-9]/*
  !sudo rm seti-breakthrough-listen.zip

  tmp_DT = np.concatenate([valid_DT, as_numpy_dataset(x_valid, is_train  = True)], axis = 0); del valid_DT
  valid_DT = np.memmap('valid_mmap', dtype = tmp_DT.dtype, mode = 'w+', shape = tmp_DT.shape)
  valid_DT[:] = tmp_DT[:]
  valid_DT.flush()
  valid_DT = valid_DT, y_valid
  del tmp_DT
  gc.collect();time.sleep(5)

  ##미리 처리된것 저장
  train_DT1 = np.memmap('tmp_train_dt1', dtype = tmp_train_DT.dtype, mode = 'w+', shape = tmp_train_DT.shape)
  train_DT1[:] = tmp_train_DT[:]
  train_DT1.flush()
  del tmp_train_DT
  gc.collect();time.sleep(5)

  ##새로 생성
  tmp_train_DT = as_numpy_dataset(x_train, is_train  = True)
  train_DT2 = np.memmap('tmp_train_dt2', dtype = tmp_train_DT.dtype, mode = 'w+', shape = tmp_train_DT.shape)
  train_DT2[:] = tmp_train_DT[:]
  train_DT2.flush()
  del tmp_train_DT
  gc.collect();time.sleep(5)

  train_DT = np.concatenate([train_DT1, train_DT2], axis = 0)
  !sudo rm -r train 

  train_DT_mmap = np.memmap('train_mmap', dtype = train_DT.dtype, mode = 'w+', shape = train_DT.shape)
  train_DT_mmap[:] = train_DT[:]
  train_DT_mmap.flush()
  del train_DT, train_DT1, train_DT2
  !sudo rm -r tmp_train_dt1
  !sudo rm -r tmp_train_dt2
  train_DT = train_DT_mmap, y_train

## tf.data.Dataset setting

In [ ]:
BATCH_SIZE = 16
train_dataset = tf.data.Dataset.from_tensor_slices(train_DT).shuffle(512).batch(BATCH_SIZE, True).prefetch(tf.data.experimental.AUTOTUNE)
valid_dataset = tf.data.Dataset.from_tensor_slices(valid_DT).batch(BATCH_SIZE).prefetch(33)
test_dataset  = tf.data.Dataset.from_tensor_slices(tests_DT).batch(BATCH_SIZE).prefetch(32)

# model define

In [ ]:
ACT = (lambda X: X * tf.keras.activations.tanh(tf.keras.activations.softplus(X)))
ACTL = tf.keras.layers.Lambda(ACT)
ACT_cosnorm = lambda x: tf.keras.activations.elu(x, alpha=0.05)

@tf.function
def make_norm(Val, axis):
    mean = tf.reduce_mean(tf.stop_gradient(Val), axis, True)
    return (Val - mean) / (1e-6 + tf.norm((Val - mean),2, axis, True) )

class coscnn(tf.keras.layers.Layer):
    def __init__(self, d, kk, ss, padding = 'VALID'):
        super(coscnn, self).__init__()
        self.args = [[1,*kk,1], [1,*ss,1], [1,1,1,1], padding]
        self.num_outputs = d
    def build(self, input_shape):
        last_shape = int(input_shape[-1]) * self.args[0][-3] * self.args[0][-2]
        self.kernel = self.add_weight(
            shape=(last_shape, self.num_outputs), initializer="he_normal", trainable=True
        )
        self.bias = self.add_weight(shape=(self.num_outputs,), initializer="zeros", trainable=True)
        if len(input_shape) == 4:
          self.extracter = lambda inputs : tf.image.extract_patches(inputs, *self.args)
        if len(input_shape) == 5:
          self.extracter = TimeDistributed(Lambda(lambda inputs : tf.image.extract_patches(inputs, *self.args)))
    def call(self, inp):
        return make_norm(self.extracter(inp), -1) @ make_norm(self.kernel, 0) + self.bias

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
wide = .781
X = IN = Input((3, 273, 256))
X = tf.expand_dims(X, -1)
X = coscnn(128, (13, 8), (13, 8), 'VALID')(X)#21x32
print(X.shape)
X = ACT_cosnorm(X)

Drops = Dropout(0.25)(tf.reduce_mean(tf.ones_like(X), -1, True))#pixel wise dropout
X = X * Drops

X = tf.concat(tf.unstack(X, axis = 1), axis = -1)

X = Conv2D(int(512 * wide), 3, 1, padding = 'same', use_bias = False, kernel_initializer = 'he_normal')(X)
X = ACT(BatchNormalization()(X))
X = Conv2D(int(256 * wide), 3, 1, padding = 'same', use_bias = False, kernel_initializer = 'he_normal')(X)
X = ACT(BatchNormalization()(X))
X = Conv2D(int(512 * wide), 3, 1, padding = 'same', use_bias = False, kernel_initializer = 'he_normal')(X)
X = ACT(BatchNormalization()(X))
X = AveragePooling2D(padding = 'same')(X)#11x16


X = Conv2D(int(1024 * wide), 3, 1, padding = 'same', use_bias = False, kernel_initializer = 'he_normal')(X)
X = ACT(BatchNormalization()(X))
X = Conv2D(int(512 * wide), 3, 1, padding = 'same', use_bias = False, kernel_initializer = 'he_normal')(X)
X = ACT(BatchNormalization()(X))
X = Conv2D(int(1024 * wide), 3, 1, padding = 'same', use_bias = False, kernel_initializer = 'he_normal')(X)
X = ACT(BatchNormalization()(X))
X = AveragePooling2D(padding = 'same')(X)#6x8
X = Flatten()(X)

X = Dense(1, dtype='float32', use_bias = False)(X)
X = Activation('sigmoid', dtype='float32')(X)
model = Model(IN, X)
model.summary()

EPOCHS = 20
class FCA(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, total_step, initial_learning_rate):
        self.total_step = total_step
        self.initial_learning_rate = initial_learning_rate

    def __call__(self, step):
        return tf.where(
            tf.less(self.total_step * 72 / 100, step)
            ,self.initial_learning_rate * (1-tf.cos(((step - self.total_step) / (self.total_step * 28 / 100) * 3.1415926535)))/2
            ,self.initial_learning_rate
        )

def loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred, from_logits=False, label_smoothing=0.1)
total_run = y_train.shape[0] // BATCH_SIZE * EPOCHS

opt = tfa.optimizers.RectifiedAdam(FCA(total_run, 0.001), weight_decay=0.0005, clipnorm = 0.01, total_steps = total_run, warmup_proportion=0.02)
opt = tfa.optimizers.Lookahead(opt, sync_period=6, slow_step_size=0.5)
opt = mixed_precision.LossScaleOptimizer(opt, loss_scale='dynamic')
model.compile(optimizer=opt, loss=loss, metrics=[keras.metrics.AUC()])

In [ ]:
def auc_plot(auc,val_auc):
    plt.plot(auc)
    plt.plot(val_auc)
    plt.xlabel('epochs')
    plt.ylabel('auc')
    plt.title('auc vs epochs')
    plt.legend(['auc','val_auc'])
    plt.show()
def loss_plot(loss,val_loss):
    plt.plot(loss)
    plt.plot(val_loss)
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.title('loss vs epochs')
    plt.legend(['loss','val_loss'])
    plt.show()

In [ ]:
history = model.fit(train_dataset,epochs=EPOCHS,validation_data=valid_dataset)

# cosine normalizing filter result

In [ ]:
filters = model.layers[2].kernel
f_min, f_max = tf.reduce_min(filters), tf.reduce_max(filters)
print(f_min, f_max)
filters = (filters - f_min) / (f_max - f_min)
n_filters, ix = 64, 1
fig, axss = plt.subplots(8, 8, figsize=(15,15))
for i, ax in zip(range(n_filters), [J for I in axss for J in I]):
    # get the filter
    f = filters[:, i]
    f = tf.reshape(f, (13, 8))
    ax.set_xticks([])
    ax.set_yticks([])
    # plot filter channel in grayscale
    ax.imshow(f, cmap='gray')
# show the figure
plt.show()

In [ ]:
auc_plot(history.history['auc'],history.history['val_auc'])
loss_plot(history.history['loss'],history.history['val_loss'])

# output distribution

In [ ]:
import seaborn as sns
@tf.function
def pred(img):return model(img, training = False)

In [ ]:
result = [[],[]]
for img, labs in train_dataset.take(33):
  for I, lab in zip(pred(img), labs):  
    result[lab].append(I)
ax = sns.distplot(result[0], bins = 10, label = 0, color = sns.color_palette("RdBu", 9)[1])
ax = sns.distplot(result[1], bins = 10, label = 1, color = sns.color_palette("RdBu", 9)[-2])

In [ ]:
result = [[],[]]
for img, labs in valid_dataset.take(33):
  for I, lab in zip(pred(img), labs):  
    result[lab].append(I)
ax = sns.distplot(result[0], bins = 10, label = 0, color = sns.color_palette("RdBu", 9)[1])
ax = sns.distplot(result[1], bins = 10, label = 1, color = sns.color_palette("RdBu", 9)[-2])

In [ ]:
preds = model.predict(test_dataset)

In [ ]:
preds = preds.reshape(-1)

In [ ]:
submission = pd.DataFrame({'id':sample_submission['id'],'target':preds})

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
#preds = model.predict(test_dataset)
#preds = preds.reshape(-1)
plt.figure(figsize=(10,5))
plt.hist(submission.target,bins=100, range = [0,1]);

# submit

In [ ]:
!kaggle competitions submit -c seti-breakthrough-listen -f submission.csv -m "from colab - training result"

# References

### https://www.kaggle.com/assign
### https://www.kaggle.com/assign/seti-with-colab-pro-gpu-tensorflow